# Setting up a new instance

Just ssh into the instance. Clone the AWS_POC repository and run two helper scripts that I've created, setup.sh and unic.sh.

Well, so far so good. We have spawned a powerful instance for a cheaper price. Let's check out the cpu, ram and storage.

In [8]:
lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              64
On-line CPU(s) list: 0-63
Thread(s) per core:  2
Core(s) per socket:  16
Socket(s):           2
NUMA node(s):        2
Vendor ID:           GenuineIntel
CPU family:          6
Model:               79
Model name:          Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz
Stepping:            1
CPU MHz:             1797.172
CPU max MHz:         3000.0000
CPU min MHz:         1200.0000
BogoMIPS:            4600.13
Hypervisor vendor:   Xen
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            46080K
NUMA node0 CPU(s):   0-15,32-47
NUMA node1 CPU(s):   16-31,48-63
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall nx pdpe1gb rdtscp lm constant_tsc arch_perfmon rep_good nopl xtopology nonstop_tsc cpuid aperfmperf pni pclmulqdq monito

In [9]:
free -h

              total        used        free      shared  buff/cache   available
Mem:           480G        727M        472G        1.0M        7.1G        476G
Swap:            0B          0B          0B


In [10]:
lsblk

NAME    MAJ:MIN   RM  SIZE RO TYPE MOUNTPOINT
loop0     7:0      0 87.9M  1 loop /snap/core/5328
loop1     7:1      0 12.7M  1 loop /snap/amazon-ssm-agent/495
xvda    202:0      0    8G  0 disk 
└─xvda1 202:1      0    8G  0 part /
xvdba   202:13312  0  200G  0 disk 


Very good! Remember that we added an extra 200 Gb - that's xvdba. However,  it is not yet mounted - see the mountpoint.

In [12]:
sudo file -s /dev/xvdba

/dev/xvdba: data


As shown above, the EBS has no filesystem yet. Otherwise, it would show more information about the filesystem. Make the filesystem.

In [13]:
sudo mkfs.ext4 /dev/xvdba

mke2fs 1.44.1 (24-Mar-2018)
Creating filesystem with 52428800 4k blocks and 13107200 inodes
Filesystem UUID: 12e1f0e1-81fb-4bb6-9e22-59fd5f72bfef
Superblock backups stored on blocks: 
	32768, 98304, 163840, 229376, 294912, 819200, 884736, 1605632, 2654208, 
	4096000, 7962624, 11239424, 20480000, 23887872

Allocating group tables:    0/160        done                            
Writing inode tables:    0/160        done                            
Creating journal (262144 blocks): done
Writing superblocks and filesystem accounting information:    0/160        done



Now, create a directory, then mount the drive on to it.

In [17]:
mkdir EColi
sudo mount /dev/xvdba EColi

In [19]:
lsblk

NAME    MAJ:MIN   RM  SIZE RO TYPE MOUNTPOINT
loop0     7:0      0 87.9M  1 loop /snap/core/5328
loop1     7:1      0 12.7M  1 loop /snap/amazon-ssm-agent/495
xvda    202:0      0    8G  0 disk 
└─xvda1 202:1      0    8G  0 part /
xvdba   202:13312  0  200G  0 disk /home/ubuntu/EColi


Okay, good. We now have a directory called EColi. That is where we are going to do all our work with Unicycler. Remember about the 8Gb xvda drive? Well, all the other prior softwares, like Anaconda, were installed there - just so you know.

Everytime we reboot this instance, we will need to repeat this ritual of mounting this drive - which is tedious. Simply add the line to our .bashrc file. For some reason, you need to sudo this, otherwise, only xvda1 would be shown.

In [25]:
sudo blkid

/dev/xvda1: LABEL="cloudimg-rootfs" UUID="bbf64c6d-bc15-4ae0-aa4c-608fd9820d95" TYPE="ext4" PARTUUID="9f3e4931-01"
/dev/loop0: TYPE="squashfs"
/dev/loop1: TYPE="squashfs"
/dev/xvdba: UUID="12e1f0e1-81fb-4bb6-9e22-59fd5f72bfef" TYPE="ext4"


Edit the /etc/fstab file - there is where supposedly all the mounting is listed. The fields are the UUID value returned by blkid, the mount point, the file system, and the recommended file system mount options (copied from [AWS](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/ebs-using-volumes.html)). Here is how it looks like after adding our device.

In [26]:
cat /etc/fstab

LABEL=cloudimg-rootfs	/	 ext4	defaults,discard	0 0
UUID=12e1f0e1-81fb-4bb6-9e22-59fd5f72bfef  /home/ubuntu/EColi  ext4  defaults,nofail  0  2


This is how to check if it works. If it isn't we will receive an error message.

In [28]:
sudo umount /home/ubuntu/EColi
sudo mount -a

Nice. Next, we'll continue with Unicycler.